In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt
import pyomo.environ as pe

from scipy.stats import rv_histogram

import nice
from nice.reload import deep_reload

In [2]:
'''
Loading
'''

# atlas = nice.graph.graph_from_json('Data/reduced_atlas.json')

# graph_all = nice.graph.graph_from_json('Data/graph_100k.json')
graph_ccs = nice.graph.graph_from_json('Data/graph_ccs_100k.json')
# graph_nacs = nice.graph.graph_from_json('Data/graph_nacs_100k.json')

In [3]:
'''
Selecting
'''

graph = graph_ccs

for source, _adj in graph._adj.items():
    for target, edge in _adj.items():

        edge['cost'] = edge['time']

places = [k for k, n in graph._node.items() if 'station' not in k]
stations = [k for k, n in graph._node.items() if 'station' in k]

In [4]:
'''
Raw Paths
'''
deep_reload(nice)

raw_paths = nice.graph.get_raw_paths(
    graph, places[:10],
    objective =  'distance',
    min_edge_cost = 150e3,
)

 [-------------------->] 100.00%, 0.36 it/s, 30.90 s elapsed, 0.00 s remaining

In [8]:
len(raw_paths), len(places) ** 2 * 100

(538162, 562500)

In [321]:
deep_reload(nice)

sp = nx.algorithms.simple_paths._all_simple_edge_paths

s = 'Sacramento, CA'
t = 'Los Angeles, CA'
t1 = 'Fresno, CA'

terminals = list(set(places) - set([s]))

lg = nice.graph.level_graph(
    graph, s, objective = 'distance',
    min_edge_cost = 150e3,
    terminals = terminals,
)

print(lg.number_of_edges())

asp = list(sp(lg, s, set(terminals), cutoff = 4))
# asp1 = list(nx.all_simple_paths(lg, s, t1, cutoff = 4))

48158


In [320]:
len(asp)

388104

In [334]:
deep_reload(nice)

def make_paths(graph, raw_paths, ac_power = 6600):

    paths = []

    for raw_path in nice.progress_bar.ProgressBar(raw_paths, freq = 1e4):

        tail, head = raw_path[0]

        path = {
            'path': [raw_path[0][0]] + [e[1] for e in raw_path],
            'cost_dc': graph._adj[tail][head]['time'],
            'cost_ac': graph._adj[tail][head]['time'],
        }

        for tail, head in raw_path[1:]:

            path['cost_dc'] += (
                graph._adj[tail][head]['time'] +
                graph._adj[tail][head]['energy'] /
                (graph._node[tail]['power_kw'] * 1e3)
            )

            path['cost_ac'] += (
                graph._adj[tail][head]['time'] +
                graph._adj[tail][head]['energy'] / ac_power
            )

        paths.append(path)

    return paths

paths = make_paths(graph, asp)

 [---------------->] 100.00%, 405020.88 it/s, 0.96 s elapsed, 0.00 s remaining

In [12]:
raw_paths[-20:]

[[('Sacramento, CA', 'San Diego, CA')],
 [('Sacramento, CA', 'Richmond, CA')],
 [('Sacramento, CA', 'Ontario, CA')],
 [('Sacramento, CA', 'El Cajon, CA')],
 [('Sacramento, CA', 'Oceanside, CA')],
 [('Sacramento, CA', 'Berkeley, CA')],
 [('Sacramento, CA', 'Fremont, CA')],
 [('Sacramento, CA', 'Carlsbad, CA')],
 [('Sacramento, CA', 'Fontana, CA')],
 [('Sacramento, CA', 'Rialto, CA')],
 [('Sacramento, CA', 'Murrieta, CA')],
 [('Sacramento, CA', 'Clovis, CA')],
 [('Sacramento, CA', 'Fresno, CA')],
 [('Sacramento, CA', 'Antioch, CA')],
 [('Sacramento, CA', 'Concord, CA')],
 [('Sacramento, CA', 'Salinas, CA')],
 [('Sacramento, CA', 'Santa Maria, CA')],
 [('Sacramento, CA', 'El Monte, CA')],
 [('Sacramento, CA', 'Pasadena, CA')],
 [('Sacramento, CA', 'Elk Grove, CA')]]